# Feature Extraction

In [19]:
import os

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats

import activity_classifier_utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the data

In [4]:
fs = 256
data = activity_classifier_utils.LoadWristPPGDataset()

### Features
Time Domain:
* mean
* std
* 5, 10, 15, 20, 25 percentile
* cross-correlation of all pairs of channels
* total energy

Frequency Domain:
* dominant frequency
* fraction of energy in each 1Hz bin from 0 to 6 Hz
* spectral entropy of each channel - i'll do

Low-pass filter at 12 Hz

In [6]:
def LowpassFilter(signal, fs, cutOffFreq):
    b, a = sp.signal.butter(3, cutOffFreq, btype='lowpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

Compute Features

In [49]:
def Featurize(accx, accy, accz, fs):
    """A partial featurization of the accelerometer signal.
    
    Args:
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate of the accelerometer
        
    Returns:
        n-tuple of accelerometer features
    """
    
    accx = LowpassFilter(accx, fs, cutOffFreq=12)
    accy = LowpassFilter(accy, fs, cutOffFreq=12)
    accz = LowpassFilter(accz, fs, cutOffFreq=12)
    
    # The mean of the x-channel
    mn_x = np.mean(accx)

    # The standard deviation of the x-channel
    std_x = np.std(accx)

    # The 5th percentile of the x-channel
    p5_x = np.percentile(accx, 5)

    # The pearson correlation coefficient between the x and y channels
    corr_xy = np.corrcoef(accx, accy)[0,1]

    # The total AC energy of the x-axis
    energy_x = np.sum(np.square(accx - np.mean(accx)))
    
    # Take an FFT of the signal. If the signal is too short, 0-pad it so we have at least 2046 points in the FFT.
    fft_len = max(len(accx), 2046)
    
    # Create an array of frequency bins
    fft_freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    
    # Take an FFT of the centered signal
    fft_x = np.fft.rfft(accx - np.mean(accx), fft_len)
    
    # The frequency with the most power between 0.25 and 12 Hz 
    indx_window = (fft_freqs>0.25) & (fft_freqs<12)
    dominant_frequency_x = fft_freqs[np.argmax((fft_x[indx_window]))]

    # The fraction of energy between 2 and 3 Hz in the x-channel
    spectral_energy_x = np.square(np.abs(fft_x))
    energy_23_x = np.sum(spectral_energy_x[(fft_freqs >= 2) & (fft_freqs <= 3)]) / np.sum(spectral_energy_x)
    
    return (mn_x,
            std_x,
            p5_x,
            corr_xy,
            energy_x,
            dominant_frequency_x,
            energy_23_x)

## Check The Code

Extract a 10 second window of the DataFrame

In [50]:
seg = data[0][2].iloc[:fs * 10]

In [51]:
accx = seg.accx.values
accy = seg.accy.values
accz = seg.accz.values

In [52]:
Featurize(accx, accy, accz, fs)

(2.401101684154747,
 0.37551422217316954,
 1.900888154276325,
 0.8795605831174856,
 360.98798349906065,
 2.4000000000000004,
 0.04473292806734677)

In [53]:
pd.DataFrame({'Feature names':np.array(activity_classifier_utils.FeatureNames()), 'Values':activity_classifier_utils.Featurize(accx,accy,accz,fs)})

,Feature names,Values
0,mn_x,2.401102
1,mn_y,-2.990595
2,mn_z,8.727179
3,std_x,0.375514
4,std_y,0.432756
5,std_z,0.199464
6,p5_x,1.900888
7,p5_y,-3.558525
8,p5_z,8.421095
9,p10_x,1.958165


**Note:** we did get successfully the freatures. However we did the process for only accelerometer 'x'. Therefore we should do it for all of them and add some other features with the same logic. For instance percentile 10, 25 and so for. We did all of this inside the function `activity_classifier_utils` and manage to get 54 features that now we can use to feed to a classifier. 

## Feature extraction

Now we can extract the features for all of our data.

Train on 10 second long non-overlapping windows

In [54]:
# defining window lenght and the ship which will be the same since we don't want overlapping
window_length_s = 10
window_shift_s = 10

In [57]:
# Extracting the features, labels and subjects of all our data
window_length = window_length_s * fs
window_shift = window_shift_s * fs
labels, subjects, features = [], [], []
for subject, activity, df in data:
    for i in range(0, len(df) - window_length, window_shift):
        window = df[i: i + window_length]
        accx = window.accx.values
        accy = window.accy.values
        accz = window.accz.values
        features.append(activity_classifier_utils.Featurize(accx, accy, accz, fs=fs))
        labels.append(activity)
        subjects.append(subject)
labels = np.array(labels)
subjects = np.array(subjects)
features = np.array(features)

In [76]:
labels_df = pd.DataFrame({'labels':labels})
print(f"size of the labels: {labels_df.shape[0]}")
labels_df.groupby(['labels']).first()

size of the labels: 611


""
labels
bike
run
walk


In [75]:
subjects = pd.DataFrame({'subjects':subjects})
print(f"size of the subjects: {subjects.shape[0]}")
subjects.groupby(['subjects']).first()

size of the subjects: 611


""
subjects
s1
s2
s3
s4
s5
s6
s8
s9


In [129]:
features_df = pd.DataFrame({'features':np.zeros(611)})
features_df["features"] = features.tolist()
print(f"size of the features: {features_df.shape[0]}")
features_df.head()

size of the features: 611


,features
0,"[2.401101684154747, -2.9905948346879363, 8.727..."
1,"[3.034925716596861, -3.3382791114863877, 8.422..."
2,"[3.0790489022228793, -3.807686227657727, 8.216..."
3,"[1.766370410389475, -3.128901028859688, 8.7495..."
4,"[1.6972330156487523, -3.074743436453803, 8.832..."


We started with 10 seconds of 256 Hz accelerometer data. That's 2500 samples per channel, and for three channel that's 7500 points. We've successfully reduced these 7500 points to just 55 points while hopefully retaining all the information we need to build a good classifier.

Although we only have 8 subjects of data, we have 611 datapoints because each 10 second window is its own datapoint. However, our datapoints are not independent. Because there's homogeneity in how individuals do an activity, datapoints from the same person might be more similar to each other. We have to keep this in mind when we train and evaluate our model. In the next video we'll use these features to build a random forest model and classify our data.